# Detecting road lane lines in images and videos

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os, glob
import numpy as np
from moviepy.editor import VideoFileClip

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The images and videos used in this project are taken from Udacity dataset on github.

In [ ]:
!git clone https://github.com/udacity/CarND-LaneLines-P1.git

Defining a helper function for displaying images.

In [ ]:
def show_images(images, cmap=None):
    cols = 2
    rows = (len(images)+1)//cols
    
    plt.figure(figsize=(10, 11))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        # use gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape)==2 else cmap
        plt.imshow(image, cmap=cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [ ]:
test_images = [plt.imread(path) for path in glob.glob('/content/CarND-LaneLines-P1/test_images/*.jpg')]

show_images(test_images)

The lines in the images are white and yellow, to make things easier we'd like to extract only these colors. We'll experiment with different color spaces to do this and apply thresholding, to extract just the colors closely matching the lanes.

In [ ]:
def yellow_white_selection(image):
  lowerbw = np.uint8([200, 200, 200]) #lower threshold for white color
  upperbw = np.uint8([255, 255, 255]) #upper threshold for white color
  white_mask = cv2.inRange(image, lowerbw, upperbw) 

  lowerby = np.uint8([190, 190, 0]) #lower threshold for yellow
  upperby = np.uint8([255, 255, 255]) #upper threshold for yellow
  yellow_mask = cv2.inRange(image, lowerby, upperby)

  #combining the two masks using bitwise operations
  mask = cv2.bitwise_or(white_mask, yellow_mask)
  masked = cv2.bitwise_or(image,image, mask=mask)

  return masked

show_images(list(map(yellow_white_selection, test_images)))



Almost all the lanes are visible except for those shaded by an object. We may need to use a different color space and try to extract the lines then.

HLS (Hue, Light, Saturation) color space

In [ ]:
def to_hls(image):
  return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
show_images(list(map(to_hls, test_images)))


Now, all the lanes are clearly visible. The lanes changes colors - white to green and yellow to blue. We need to take it into consideration when defining a function for extracting these lines to make masks.

In [ ]:
def yellow_white_hls_selection(image):
  converted = to_hls(image)
  lowerbw = np.uint8([0, 200, 0]) #white lower boundary
  upperbw = np.uint8([255, 255, 255]) #white upper boundary
  white_mask = cv2.inRange(converted, lowerbw, upperbw)

  lowerby = np.uint8([10, 0, 100]) #yellow lower boundary
  upperby = np.uint8([40, 255, 255]) #yellow upper boundary (we can experiment with these)
  yellow_mask = cv2.inRange(converted, lowerby, upperby)

  #Combining the two masks by bitwise
  mask = cv2.bitwise_or(white_mask, yellow_mask)
  masked_hls = cv2.bitwise_and(image, image, mask=mask)
  return masked_hls

hls_images = list(map(yellow_white_hls_selection, test_images))
show_images(hls_images)



Canny Edge Detection


*   convert images into gray scale
*   smooth the edges with Gaussian Blur
*   use cv2.Canny() for detecting the edges






HLS images into gray scale

In [ ]:
def to_gray(image):
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  return gray

hls_to_gray_images = list(map(to_gray, hls_images))
show_images(hls_to_gray_images)


# Smoothing the edges

`cv2.GaussianBlur()` function requires `kernel_size` parameter, the number must be odd. The bigger the number, the blurrier the image.

In [ ]:
def smoothing(image, kernel_size=5):
  smoothed = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
  return smoothed

smoothed_images = list(map(smoothing, hls_to_gray_images))
show_images(smoothed_images)

# Canny Edge Detection

The `Canny` function takes two thresholds as parameters. To find their best values we need to remember:


*   If a pixel gradient is higher than the upper threshold, the pixel is accepted as an edge.
*   If a pixel gradient value is below the lower threshold, then it is rejected.
*   If the pixel gradient is between the two thresholds, then it will be accepted only if it is connected to a pixel that is above the upper threshold.
*   Canny recommended the upper:lower ratio between 2:1 and 3:1.
*   If the high threshold is too high, we'll see no edges.
*   If the low threshold is too low, we'll see too much noise

In [ ]:
def canny_edges(image, low_threshold = 90, high_threshold=180):
  edges = cv2.Canny(image, low_threshold, high_threshold)
  return edges

canny_images = list(map(canny_edges, smoothed_images))
show_images(canny_images)

# Selecting the region of interest

After we've detected the edges in the picture we're going to to choose the region of interest by contouring the road. We'll specify the vertices surrounding the road and only use this region for lane detection. This way we'll get rid of cannied cars, trees etc. 

In [ ]:
def region_of_interest(image, vertices):
  copy = np.zeros_like(image)
  if len(copy.shape)==2:
    cv2.fillPoly(copy, vertices, 255)
  else:
    cv2.fillPoly(copy, vertices, (255,)*copy.shape[2]) # in case, the input image has a channel dimension        
  return cv2.bitwise_and(image, copy)

def vertices(image):

  # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.1, rows*0.95]
    top_left     = [cols*0.4, rows*0.6]
    bottom_right = [cols*0.9, rows*0.95]
    top_right    = [cols*0.6, rows*0.6] 
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return region_of_interest(image, vertices)

region_images = list(map(vertices, canny_images))
show_images(region_images)

# Hough Transform
Now we need to recongnize the lanes in an image. To do this we'll use `cv2.HoughLinesP()`. 

In [ ]:
def hough_lines(image):
  # Define the Hough transform parameters
  # image is the output of our region selection
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 300    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)
    return lines

list_of_lines = list(map(hough_lines, region_images ))

# Drawing detected lines into the image

Function `hough_lines` gives us a list of lines, not an image. To see them in an image we need to draw them first.

In [ ]:
def draw_lines(image, lines, color=(255, 0, 0), thickness=2):
  image = np.copy(image)
  # Iterate over the output "lines" and draw lines on a blank image
  for line in lines:
      for x1,y1,x2,y2 in line:
           cv2.line(image,(x1,y1),(x2,y2),color, thickness)
  return image

line_images = []
for image, lines in zip(test_images, list_of_lines):
    line_images.append(draw_lines(image, lines))

show_images(line_images)

# Extrapolating the lanes
The lanes in the pictures are not continuous. We'll extrapolate them. Each line is a function. So, first we'll define what is left and right lane. The feature that enables us to do this is the slope of a function. If the slope is > than 0, the function is positive, if the slope < 0 it is negative. Since the pixel matrix is reveresed (it's a mirror reflection by the x axis), left line will be negative, and right one positive.

In [ ]:
def average_slope_intercept(lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane, right_lane # (slope, intercept), (slope, intercept)

Now, we need to convert a line represented in slope and intercept into pixel points.

In [ ]:
def make_line_points(y1, y2, line):
    if line is None:
        return None
    
    slope, intercept = line
    
    # make sure everything is integer as cv2.line requires it
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))

Now, we need to connect the dots and draw extrapolated lines in the image.

In [ ]:
def lane_lines(image, lines):
    left_lane, right_lane = average_slope_intercept(lines)
    
    y1 = image.shape[0] # bottom of the image
    y2 = y1*0.6         # slightly lower than the middle

    left_line  = make_line_points(y1, y2, left_lane)
    right_line = make_line_points(y1, y2, right_lane)
    
    return left_line, right_line

    
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=15):
    # make a separate image to draw lines on
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    # image1 * α + image2 * β + λ
    # image1 and image2 must be the same shape.
    final = cv2.addWeighted(image, 1.0, line_image, 0.95, 0.0)
    return final
             
    
lane_images = []
for image, lines in zip(test_images, list_of_lines):
    lane_images.append(draw_lane_lines(image, lane_lines(image, lines)))

    
show_images(lane_images)

All functions for lane detection have been defined. `image_pipeline` will combine them all together. It takes three parameters: image, image_path and outpath. We'll need paths to lated save an image in a folder.

In [ ]:
#Working Pipline
def image_pipeline(image, image_path, outpath):
  hls_image = yellow_white_hls_selection(image)
  hls_to_gray_image = to_gray(hls_image)
  smoothed_image = smoothing(hls_to_gray_image)
  canny_image = canny_edges(smoothed_image)
  region_image = vertices(canny_image)
  lines = hough_lines(region_image)
  left_line, right_line = lane_lines(image, lines)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  final = draw_lane_lines(image, (left_line, right_line))
  my_dpi=96
  plt.figure(figsize=(960/my_dpi, 540/my_dpi), dpi=my_dpi)
  final_image = plt.imshow(final)
  plt.axis('off')
  save_fname = os.path.join(outpath, os.path.basename(image_path))
  plt.savefig(save_fname, bbox_inches='tight', pad_inches=0, transparent=True)

Testing the pipeline.

In [ ]:
image = cv2.imread('/content/CarND-LaneLines-P1/test_images/solidWhiteRight.jpg')
image_path = '/content/CarND-LaneLines-P1/test_images/solidWhiteRight.jpg'
outpath = '/content/CarND-LaneLines-P1'
image_pipeline(image, image_path, outpath)

# Defining helper functions for processing videos.

`clip_to_frame` takes any video and divides it into seperate frames.

In [ ]:
def clip_to_frame(vid_path, save_path):
  vidcap = cv2.VideoCapture(vid_path)
  success,image = vidcap.read()
  count = 100
  while success:
    cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file 
    os.chdir(save_path)     
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

`clip_to_detected` incorporates `clip_to_frame` and `image_pipeline` for all frames in a given video.

In [ ]:
def clip_to_detected(vid_path, save_path, detected_path):
  clip_to_frame(vid_path, save_path)
  count= 0
  for file in os.listdir(save_path):
    image = cv2.imread(file)
    image_pipeline(image, file, detected_path)
    count=+1

# Testing the method on a video.

In [ ]:
parent= '/content/CarND-LaneLines-P1'
os.mkdir(os.path.join(parent, 'Frames White Right'))
os.mkdir(os.path.join(parent, 'White Right Detected'))
vid_path='/content/CarND-LaneLines-P1/test_videos/solidWhiteRight.mp4'
save_path='/content/CarND-LaneLines-P1/Frames White Right'
out_path='/content/CarND-LaneLines-P1/White Right Detected'

clip_to_detected(vid_path, save_path, out_path)

Make video out of images with detected lane lines and save into a folder.

In [ ]:
import cv2
import os
import natsort

sorted(os.listdir('/content/CarND-LaneLines-P1/White Right Detected'))

image_folder = '/content/CarND-LaneLines-P1/White Right Detected'
video_name = '/content/CarND-LaneLines-P1/video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
natsort.natsorted(images)
images.sort()
frame = cv2.imread(os.path.join(image_folder, images[1]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 30, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()